# EDA: nq_open_gold

The goal here is to explore the gold standard dataset from the [Power of Noise](https://github.com/florin-git/The-Power-of-Noise) \\
The description is available [here](https://huggingface.co/datasets/florin-hf/nq_open_gold)
What will be the exploration:
- What are the column and size of the dataset?
- How to relate the gold document with the WikiDump2018?

In [5]:
import polars as pl
from datasets import load_from_disk, load_dataset


In [7]:
PATH = "../data"
golden =load_dataset("json", data_files=f"{PATH}/nq_open_gold/test_dataset.json")

Generating train split: 2889 examples [00:00, 20635.70 examples/s]


In [ ]:
golden["train"]

{'example_id': -3290814144789249484,
 'question': 'who got the first nobel prize in physics',
 'answers': ['Wilhelm Conrad Röntgen'],
 'text': 'The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen , of Germany , who received 150,782 SEK , which is equal to 7,731,004 SEK in December 2007 . John Bardeen is the only laureate to win the prize twice -- in 1956 and 1972 . Maria Skłodowska - Curie also won two Nobel Prizes , for physics in 1903 and chemistry in 1911 . William Lawrence Bragg was , until October 2014 , the youngest ever Nobel laureate ; he won the prize in 1915 at the age of 25 . Two women have won the prize : Curie and Maria Goeppert - Mayer ( 1963 ) . As of 2017 , the prize has been awarded to 206 individuals . There have been six years in which the Nobel Prize in Physics was not awarded ( 1916 , 1931 , 1934 , 1940 -- 1942 ) .',
 'idx_gold_in_corpus': 20994698}